In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import logging

# Set up logging to capture outputs
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Load datasets
olid_train = pd.read_csv('/content/drive/MyDrive/sem4/sem4/olid-train-small.csv')
hasoc_train = pd.read_csv('/content/drive/MyDrive/sem4/sem4/hasoc-train.csv')
olid_test = pd.read_csv('/content/drive/MyDrive/sem4/sem4/olid-test.csv')

# Prepare OLID and HASOC train data
olid_train_data = pd.DataFrame({
    'text': olid_train['text'],
    'labels': olid_train['labels']
})

hasoc_train_data = pd.DataFrame({
    'text': hasoc_train['text'],
    'labels': hasoc_train['labels']
})

# Prepare OLID test data
olid_test_data = pd.DataFrame({
    'text': olid_test['text'],
    'labels': olid_test['labels']
})

# Define the models to experiment with
models_to_try = [
    ("bert", "bert-base-cased"),
    ("roberta", "roberta-base"),
    ("bert", "GroNLP/hateBERT"),
    ("distilbert", "distilbert-base-cased"),
    ("xlnet", "xlnet-base-cased")
]

# Define the hyperparameter combinations to experiment with
hyperparameter_combinations = [
    {"learning_rate": 2e-5, "train_batch_size": 16, "warmup_steps": 500},
    {"learning_rate": 3e-5, "train_batch_size": 32, "warmup_steps": 1000},
    {"learning_rate": 1e-5, "train_batch_size": 16, "warmup_steps": 500},
]

experiment_results = []

# Run training and evaluation for one experiment
def run_experiment(model_type, model_name, hyperparams, train_data, eval_data, train_source):
    output_dir = f"/content/drive/MyDrive/sem4/outputs/{train_source}_{model_name}_lr_{hyperparams['learning_rate']}_bs_{hyperparams['train_batch_size']}_epochs_5"

    print(f"Training {model_name} on {train_source} with hyperparameters: {hyperparams}")

    # Create the model with the current set of hyperparameters
    model = ClassificationModel(
        model_type,
        model_name,
        num_labels=2,
        use_cuda=True,
        args={
            "learning_rate": hyperparams['learning_rate'],
            "num_train_epochs": 5,
            "train_batch_size": hyperparams['train_batch_size'],
            "eval_batch_size": 32,
            "warmup_steps": hyperparams['warmup_steps'],
            "output_dir": output_dir,
            "overwrite_output_dir": True,
            "reprocess_input_data": True,
            "save_steps": -1,
            "save_model_every_epoch": False
        }
    )

    model.train_model(train_data)

    result, model_outputs, wrong_predictions = model.eval_model(eval_data)

    experiment_results.append({
        "model_name": model_name,
        "train_source": train_source,
        "hyperparameters": hyperparams,
        "result": result
    })

# Loop over each model and each set of hyperparameters
for model_type, model_name in models_to_try:
    for hyperparams in hyperparameter_combinations:
        # Train on OLID, evaluate on OLID
        run_experiment(model_type, model_name, hyperparams, olid_train_data, olid_test_data, "OLID")

        # Train on HASOC, evaluate on OLID (cross-domain)
        run_experiment(model_type, model_name, hyperparams, hasoc_train_data, olid_test_data, "HASOC")